In [1]:
import pandas as pd
import numpy as np
import sqlite3 as db
from io import StringIO
from csv import writer 

In [2]:
con = db.connect('accounts.db')

In [3]:
# read all data from decay_amount table
# NOTE: The most important information we need from here is the initial_claimable_amount.

sql = 'SELECT * FROM decay_amount'
decay_amount = pd.read_sql(sql, con)

# clean empty data
decay_amount['initial_claimable_amount']=decay_amount.initial_claimable_amount.replace('',np.nan).astype(float)
decay_amount['vote_action']=decay_amount.vote_action.replace('',np.nan).astype(float)
decay_amount['ibc_action']=decay_amount.ibc_action.replace('',np.nan).astype(float)
decay_amount['delegate_action']=decay_amount.delegate_action.replace('',np.nan).astype(float)
decay_amount['evm_action']=decay_amount.evm_action.replace('',np.nan).astype(float)
decay_amount['total_lost']=decay_amount.total_lost.replace('',np.nan).astype(float)
decay_amount['total_claimed']=decay_amount.total_claimed.replace('',np.nan).astype(float)

decay_amount.count()

id                          168662
sender                      168662
vote_action                 122772
ibc_action                  106613
delegate_action             120183
evm_action                  132062
total_claimed               168662
total_lost                  155341
initial_claimable_amount    155341
total_lost_evmos            168662
dtype: int64

In [4]:
# read all claims events from claim_event table
# NOTE: This represents all claims events in the chain's history

sql = 'SELECT * FROM claim_event'
claim_event = pd.read_sql(sql, con)

claim_event['amount'] = claim_event['amount'].str.replace('aevmos', '').astype(float)
claim_event['height'] = claim_event['height'].astype(int)

claim_event.count()

id              520043
sender          520043
height          520043
amount          520043
claim_action    520043
dtype: int64

In [5]:
# read all merge events from merged_event table
# NOTE: This represents all merge events in the chain's history

sql = 'SELECT * FROM merged_event'
merge_event = pd.read_sql(sql, con)

merge_event['claimed_coins'] = merge_event['claimed_coins'].str.replace('aevmos', '').astype(float)
merge_event['fund_community_pool_coins'] = merge_event['fund_community_pool_coins'].str.replace('aevmos', '').astype(float)
merge_event['height'] = merge_event['height'].astype(int)
merge_event['sender_genesis_claim_record'] = merge_event['sender_genesis_claim_record'].astype(float)

merge_event.count()

id                             1022
recipient                      1022
sender                         1022
height                         1022
claimed_coins                  1022
fund_community_pool_coins      1022
sender_evmos_prefix            1022
sender_genesis_claim_record     950
dtype: int64

In [6]:
# Claim Events Decay loss
# Iterate over all claim events:
# 1. Find the sender in the decay_amount table
# 2. If the sender was found in the decay_amount table, check if the initial_claimable_amount is an integer and add that amount to the claimable_at_event
# 2. Find the sender in the merged_event table
# 3. If the sender was found in the merged_event table, check if the height of the merged_event is less than the height of the claim_event
# 4. If the height of the merged_event is less than the height of the claim_event, then we add the `sender_genesis_claim_record` of the merged event to the claimable_at_event amount
# 5. Calculate the lost amount
# NOTE: This process takes a long time to run as it iterates over more than 500k accounts and its probably not the most efficient way to do it.

count_progress = 0
output = []
for index, row in claim_event.iterrows():
    claimable_at_event = 0
    if count_progress == 1000:
        print('processed ', index)
        count_progress = 0
    # find in dataframe row with COLUMN = VALUE
    decay_row = decay_amount.loc[decay_amount['sender'] == row['sender']]
    initial_claimable_amount = decay_row['initial_claimable_amount'].values[0]
    if initial_claimable_amount.is_integer():
        claimable_at_event = initial_claimable_amount
    # find sender in merged_event and check if was not found
    merge_row = merge_event.loc[merge_event['recipient'] == row['sender']]
    # check if we should increase the claimable amount at that block height
    if not merge_row.empty and merge_row['height'].values[0] < row['height']:
        # TODO maybe I should check if the amount received at this height for the sender was good.
        # increasae claimable_at_event
        claimable_at_event = claimable_at_event + merge_row['sender_genesis_claim_record'].values[0]

    # calculate lost amount
    amount_to_claimed = claimable_at_event / 4
    lost_amount = amount_to_claimed - row['amount']

    output.append((row['id'], row['sender'], row['claim_action'], row['height'], initial_claimable_amount, claimable_at_event, amount_to_claimed, row['amount'], lost_amount))
    count_progress = count_progress + 1
result = pd.DataFrame(output, columns=['id', 'sender','event_type', 'height', 'initial_claimable_amount', 'claimable_at_event', 'expected_claim', 'amount_claimed', 'lost_amount'])
result.head()

processed  1000
processed  2000
processed  3000
processed  4000
processed  5000
processed  6000
processed  7000
processed  8000
processed  9000
processed  10000
processed  11000
processed  12000
processed  13000
processed  14000
processed  15000
processed  16000
processed  17000
processed  18000
processed  19000
processed  20000
processed  21000
processed  22000
processed  23000
processed  24000
processed  25000
processed  26000
processed  27000
processed  28000
processed  29000
processed  30000
processed  31000
processed  32000
processed  33000
processed  34000
processed  35000
processed  36000
processed  37000
processed  38000
processed  39000
processed  40000
processed  41000
processed  42000
processed  43000
processed  44000
processed  45000
processed  46000
processed  47000
processed  48000
processed  49000
processed  50000
processed  51000
processed  52000
processed  53000
processed  54000
processed  55000
processed  56000
processed  57000
processed  58000
processed  59000
proces

,id,sender,event_type,height,initial_claimable_amount,claimable_at_event,expected_claim,amount_claimed,lost_amount
0,1,evmos18xszt8l8slmm873l2v5w20v5tsh4l7en4d0jex,ACTION_DELEGATE,87701,9.323842e+19,9.323842e+19,2.330960e+19,2.330960e+19,0.0
1,2,evmos1v80jrjdzrc5zt3q5c8zwkawm0kqx48z84yq0kc,ACTION_VOTE,87701,7.050327e+18,7.050327e+18,1.762582e+18,1.762582e+18,0.0
2,3,evmos1p3a4h405w5vh4t0q64jsyvm4k8tyx9stujj75x,ACTION_DELEGATE,87701,2.353460e+19,2.353460e+19,5.883650e+18,5.883650e+18,0.0
3,4,evmos1rgf75fc9chvc2x8p4yaad5llxqzjhfd378z0zt,ACTION_VOTE,87701,4.009982e+19,4.009982e+19,1.002496e+19,1.002496e+19,0.0
4,5,evmos1az3xkmtpwdhjuwpg8llm2xh7p0ea8f4pjq4v4r,ACTION_DELEGATE,87701,2.170491e+19,2.170491e+19,5.426226e+18,5.426226e+18,0.0


In [7]:
# Total amount of addresses that were affected by the decay bug
impacted_events_results = result.loc[result['lost_amount'] > 0]
impacted_events_results.count()

id                          37113
sender                      37113
event_type                  37113
height                      37113
initial_claimable_amount    37087
claimable_at_event          37113
expected_claim              37113
amount_claimed              37113
lost_amount                 37113
dtype: int64

In [8]:
# Min height of impacted events
impacted_events_results.sort_values(by=['height']).head(5)

,id,sender,event_type,height,initial_claimable_amount,claimable_at_event,expected_claim,amount_claimed,lost_amount
235302,235303,evmos1al2j48j9fl4e4k8defvgc75hq0t8ehlhupl36d,ACTION_EVM,265410,3.226298e+19,3.226298e+19,8.065744e+18,6.387250e+18,1.678495e+18
235303,235304,evmos14v4n0umtecu38ca67s04u5xnajnghglk4fs23a,ACTION_DELEGATE,265426,4.277243e+19,4.277243e+19,1.069311e+19,8.467625e+18,2.225482e+18
235304,235305,evmos1545k52p32uskfp3aar7sewd6sc3spe9kgyecnw,ACTION_VOTE,265442,1.473063e+19,1.473063e+19,3.682658e+18,2.916138e+18,7.665203e+17
235305,235306,evmos1t8ct8gyv7qpnmsyk45p72eq4vnemat23fpt5sx,ACTION_DELEGATE,265443,7.050327e+18,7.050327e+18,1.762582e+18,1.395710e+18,3.668714e+17
235306,235307,evmos1ktel9ec72nd6addnfvdv2v52zjyqkvcn6f9wsh,ACTION_EVM,265452,7.050327e+18,7.050327e+18,1.762582e+18,1.395692e+18,3.668893e+17


In [9]:
# Max height of impacted events
impacted_events_results.sort_values(by=['height'], ascending=False).head(5)

,id,sender,event_type,height,initial_claimable_amount,claimable_at_event,expected_claim,amount_claimed,lost_amount
520042,520043,evmos1pu8sjzk8wwhp3272z948uajfdz0kkjf6l955jd,ACTION_EVM,5073387,7.050327e+18,7.050327e+18,1.762582e+18,5.019794e+14,1.762080e+18
520041,520042,evmos1a66kkjer33wyzhpy7dw839ltk78f4ytkp3upm4,ACTION_EVM,5072234,7.050327e+18,7.050327e+18,1.762582e+18,1.223862e+15,1.761358e+18
520032,520033,evmos1ama6mf2nn7dw9jruz9gumvs9wwvrsv6z4pcc9h,ACTION_IBC_TRANSFER,5071523,7.645915e+19,7.645915e+19,1.911479e+19,1.809251e+16,1.909670e+19
520026,520027,evmos19vra3y4cg8g7aymfv2gu242h4kcuytl2fufu25,ACTION_IBC_TRANSFER,5071272,4.485653e+19,4.485653e+19,1.121413e+19,1.161362e+16,1.120252e+19
520025,520026,evmos1kd6cpfn096t8uvf0pveatvseyarfrz2lccdgkj,ACTION_VOTE,5071204,NaN,6.871449e+20,1.717862e+20,2.307809e+17,1.715554e+20


In [10]:
# Total amount of lost tokens on Evmos denomination
impacted_events_results['lost_amount'].sum() / 1000000000000000000

158971.30615962716

In [11]:
# Generate csv file with results for impacted events sorted by address
impacted_events_results.sort_values(by=['sender']).to_csv('decay_loss_results.csv', index=False)